## LightGBM on Lending Club Dataset

In this dataset, we train an LightGBM binary classifier on the Lending Club dataset.

In [4]:
import lightgbm as lgb
import shap
import numpy as np
import pandas as pd
import onnxruntime as rt

from onnxmltools import convert_lightgbm, convert_sklearn
from onnxmltools.convert.common.data_types import FloatTensorType
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

SEED=2022

In [2]:
x, y = shap.datasets.adult()

# Train test split (7:3)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=SEED
)
d_train = lgb.Dataset(x_train, label=y_train)
d_test = lgb.Dataset(x_test, label=y_test)


In [11]:
# Read the dataset
data = pd.read_csv('../data/lending-club.csv')
y_all = data['loan_approval']
x_all = data.iloc[:, 0: data.shape[1] - 1]

# One-hot encode the categorical columns
x_all = pd.get_dummies(x_all, prefix_sep='-')

# Train test split (6:4)
x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.4, random_state=SEED
)

# Convert data frames to numpy arrays
feature_names = x_all.columns
x_train_matrix = x_train.to_numpy()
x_test_matrix = x_test.to_numpy()
y_train_array = y_train.to_numpy()
y_test_array = y_test.to_numpy()

# Create lightgbm dataset
d_train = lgb.Dataset(x_train, label=y_train)
d_test = lgb.Dataset(x_test, label=y_test)

In [25]:
params = {
    "verbose": 0,
    "learning_rate": 0.01,
    "max_bin": 512,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    # Randomly sample training data in each boosting iteration to avoid overfitting
    "subsample": 0.5,
    "min_data": 100,
    "boost_from_average": True,
}

model = lgb.train(
    params,
    d_train,
    5000,
    valid_sets=[d_test],
    early_stopping_rounds=50,
)

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001685 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's binary_logloss: 0.508791
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's binary_logloss: 0.508113
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's binary_logloss: 0.507385
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's binary_logloss: 0.506748
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's binary_logloss: 0.506072
[LightGBM] [Warning] No further splits with positive gain, b

In [26]:
y_pred = model.predict(x_train)
y_pred_label = [1 if y >= 0.5 else 0 for y in y_pred]
train_acc = accuracy_score(y_train, y_pred_label)

y_pred = model.predict(x_test)
y_pred_label = [1 if y >= 0.5 else 0 for y in y_pred]
test_acc = accuracy_score(y_test, y_pred_label)

print(f'Train accuracy: {train_acc:.4}, Test accuracy: {test_acc:.4}')

Train accuracy: 0.8183, Test accuracy: 0.7945


### Export the Model as ONNX

To use this model in Rust and the web, we export it as an ONNX model.

In [27]:
initial_types = [("float_input", FloatTensorType([None, x_train.shape[1]]))]
model_onnx = convert_lightgbm(model, initial_types=initial_types)

The maximum opset needed by this model is only 9.


In [29]:
# Save the ONNX model
with open('./lending-club-lightgbm.onnx', 'wb') as fp:
    fp.write(model_onnx.SerializeToString())

### ONNX Model Inference

Model inference using the saved ONNX model.


In [30]:
session = rt.InferenceSession('./lending-club-lightgbm.onnx')
y_pred_onnx = session.run(None, {'float_input': x_test.astype(np.float32).to_numpy()})

2023-01-30 16:51:15.234291 [W:onnxruntime:, execution_frame.cc:828 VerifyOutputSizes] Expected shape from model of {1} does not match actual shape of {2000} for output label


In [31]:
test_acc = accuracy_score(y_test, y_pred_onnx[0])
print(f'ONNX Test accuracy: {test_acc:.4}')

ONNX Test accuracy: 0.7945


### Create a Data Subset and Export CSV

In [ ]:
# data = np.load('../data/lending-club-data-5000.npz', allow_pickle=True)

# x_all = data['x_all']
# y_all = data['y_all']
# feature_names = data['feature_names']
# feature_types = data['feature_types']
# cont_index = data['cont_index']
# cat_index = data['cat_index']

# for i in cat_index:
#     counter = Counter(x_all[:, i])
#     print(i, feature_names[i], len(counter))

# # Create a data subset with only essential categorical features
# selected_feature_indexes = cont_index.tolist()
# selected_feature_indexes.extend([1, 3, 5, 10, 14, 16])

# data_df_dict = {}
# for i in selected_feature_indexes:
#     name = feature_names[i]
#     data_df_dict[name] = x_all[:, i]
# data_df_dict['loan_approval'] = y_all

# data_df = pd.DataFrame(data_df_dict)
# data_df.to_csv('../data/lending-club.csv', index=False)